# 1. Language Modeling

In this part, let's generate text using a trigram language model.

Go to https://drive.google.com/drive/folders/1pR0koayRSgXfTD72HZUHN14uec0SrnXy?usp=sharing and click add shortcut to drive. This will add the data required for this problem set to your Google drive.

<img src="https://drive.google.com/uc?id=1LqHisiziX8Ri94Xs6Cv8mhx6vivFM3kS" alt="Drawing" height="300"/>


Run the below code snippet. It will generate a URL which generates an authorization code.* Enter it below to give Colab access to your Google drive. 

*Copy function may not work. If so, manually copy the authorization code.

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


When you run the `ls` command below, you should see these folders.




In [3]:
!ls "/content/drive/My Drive/nl2ds"

semantic-parser  tweets


Let's load the trigrams first. You can change the below code as you see fit.

In [4]:
from math import log

bigram_prefix_to_trigram = {}
bigram_prefix_to_trigram_weights = {}

lines = open("/content/drive/My Drive/nl2ds/tweets/covid-tweets-2020-08-10-2020-08-21.trigrams.txt").readlines()
for line in lines:
  word1, word2, word3, count = line.strip().split()
  if (word1, word2) not in bigram_prefix_to_trigram:
    bigram_prefix_to_trigram[(word1, word2)] = []
    bigram_prefix_to_trigram_weights[(word1, word2)] = []
  bigram_prefix_to_trigram[(word1, word2)].append(word3)
  bigram_prefix_to_trigram_weights[(word1, word2)].append(int(count))

# freeup memory
lines = None

In [5]:
#print(bigram_prefix_to_trigram_weights[('middle' , 'of')])
#print(bigram_prefix_to_trigram[('middle' , 'of')])
count = 0

for n in bigram_prefix_to_trigram_weights[('middle' , 'of')]:
  count += n

l1 = bigram_prefix_to_trigram[('middle' , 'of')]
l2 = bigram_prefix_to_trigram_weights[('middle' , 'of')]

i = 0
for word,c in zip(l1,l2):
  if i == 10:
    break;
  prob = c/count
  print(word, prob)
  i += 1


a 0.807981220657277
the 0.06948356807511737
pandemic 0.023943661971830985
this 0.016901408450704224
an 0.0107981220657277
covid 0.009389671361502348
nowhere 0.008450704225352112
it 0.004694835680751174
lockdown 0.002347417840375587
summer 0.002347417840375587


## Problem 1.1: Retrieve top next words and their probability given a bigram prefix.

For the following prefixes **word1=middle, word2=of, and n=10**, the output is:



```
a 0.807981220657277
the 0.06948356807511737
pandemic 0.023943661971830985
this 0.016901408450704224
an 0.0107981220657277
...
...
...
```



In [6]:
def top_next_word(word1, word2, n=10):
  next_words = []
  probs = []
  count = 0
  for n1 in bigram_prefix_to_trigram_weights[(word1, word2)]:
    count += n1
  
  i = 0
  for word,c in zip(bigram_prefix_to_trigram[(word1,word2)], bigram_prefix_to_trigram_weights[(word1, word2)]):
    if i == n:
      break
    prob = c/count
    probs.append(prob)
    next_words.append(word)
    i += 1
  
  return next_words, probs

  


next_words, probs = top_next_word("middle", "of", 10)
for word, prob in zip(next_words, probs):
  print(word, prob)

a 0.807981220657277
the 0.06948356807511737
pandemic 0.023943661971830985
this 0.016901408450704224
an 0.0107981220657277
covid 0.009389671361502348
nowhere 0.008450704225352112
it 0.004694835680751174
lockdown 0.002347417840375587
summer 0.002347417840375587


## Problem 1.2: Sampling n words

Sample next n words given a bigram prefix. Use the probablity distribution defined by the frequency counts. Functions like **numpy.random.choice** will be useful here. Sample without repitition, otherwise all your samples will contain the most frequent trigram.


For the following prefixes **word1=middle, word2=of, and n=10**, the output could be as follows (our outputs may differ): 

```
a 0.807981220657277
pandemic 0.023943661971830985
nowhere 0.008450704225352112
the 0.06948356807511737
...
...
...
...
...
```



In [8]:
import numpy as np
def sample_next_word(word1, word2, n=10):
  next_words = bigram_prefix_to_trigram[(word1, word2)]
  probs = bigram_prefix_to_trigram_weights[(word1, word2)]

  total_count = 0
  for prob in probs:
    total_count += prob
  
  final_probs = []
  for prob in probs:
    final_probs.append(prob/total_count)
    
  #gave error during last part of q1, added for that intent
  if len(next_words) < n:
      n = len(next_words)
  sampled_words = np.random.choice(next_words, n, replace=False, p=final_probs)  
  new_probs = []
  
  for sample in sampled_words:
    for word,prob in zip(next_words, probs):
      if sample == word:
        new_probs.append(prob/total_count)
        continue

  
  return sampled_words, new_probs

next_words, probs = sample_next_word("middle", "of", 10)
for word, prob in zip(next_words, probs):
  print(word, prob)


a 0.807981220657277
oh 0.00046948356807511736
pandemic 0.023943661971830985
fifth 0.00046948356807511736
covid 0.009389671361502348
the 0.06948356807511737
an 0.0107981220657277
nowhere 0.008450704225352112
#covid 0.00046948356807511736
this 0.016901408450704224


## Problem 1.3: Generate sentences starting with a prefix

Generates n-sentences starting with a given sentence prefix. Use [beam search](https://en.wikipedia.org/wiki/Beam_search) to generate multiple sentences. Depending on which method you use to generate next word, you will get different outputs. When you generate <EOS> in a path, stop exploring that path. If you are not careful with your implementation, you may end up in an infinite loop.

If you use the method `word_generator=top_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> trump`, your output is as follows:
```
<BOS1> <BOS2> trump eyes new unproven coronavirus treatment URL <EOS> 0.00021893147502903603
<BOS1> <BOS2> trump eyes new unproven coronavirus cure URL <EOS> 0.0001719607222046247
<BOS1> <BOS2> trump eyes new unproven virus cure promoted by mypillow ceo over unproven therapeutic URL <EOS> 9.773272077557522e-05
...
...
...
```


If you use the method `word_generator=top_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> biden`, your output is as follows:
```
<BOS1> <BOS2> biden calls for a 30 bonus URL #cashgem #cashappfriday #stayathome <EOS> 0.0002495268686322749
<BOS1> <BOS2> biden says all u.s. governors should mandate masks <EOS> 1.6894510541025754e-05
<BOS1> <BOS2> biden says all u.s. governors question cost of a pandemic <EOS> 8.777606198953028e-07
...
...
...
```


If you use the method `word_generator=sample_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> trump`, your output may look as follows (since this is sampling, our outputs will difer):

```
<BOS1> <BOS2> trump signs executive orders URL <EOS> 7.150992253427233e-05
<BOS1> <BOS2> trump signs executive actions URL <EOS> 7.117242889600614e-05
<BOS1> <BOS2> trump news president attacked over it <EOS> 1.0546494007903964e-05
<BOS1> <BOS2> trump news president attacked over executive orders URL <EOS> 1.0126405114118984e-05
```

If you use the method `word_generator=sample_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> biden`, your output may look as follows:

```
<BOS1> <BOS2> biden harris 2020 <EOS> 0.0015758924114719264
<BOS1> <BOS2> biden harris 2020 URL <EOS> 0.0006443960952032196
<BOS1> <BOS2> biden calls for evictions ban so marylander 's do it URL <EOS> 4.105215709355001e-07
<BOS1> <BOS2> biden calls for evictions ban so marylander 's do our best to stay home <EOS> 1.3158806336098573e-09
...
...
...
...
...
```

Hope you see that sampling gives different outputs compared to deterministically picking the top n-words.


In [9]:
import heapq


def generate_sentences(prefix, sampler, beam=10):
  sentence = prefix.split()



  start_beam = [(1.0 , sentence , False)]
  

  while 1:
    new_beam = []
    for score, sentence,boolean in start_beam:
      end = sentence[-1]

      if end == '<EOS>':
        boolean = True
        heapq.heappush(new_beam, (score, sentence, boolean))
        if len(new_beam) > beam:
          heapq.heappop(new_beam)

      else:
        next_words, probs = sampler(sentence[-2], sentence[-1] , n=beam)
      
        for word, prob in zip(next_words, probs):
          new_score = prob * score
          heapq.heappush(new_beam, (new_score, sentence + [word] , False) )
          if len(new_beam) > beam:
            heapq.heappop(new_beam)
    start_beam = new_beam
    all_done = True
    for (_,_,boolean) in start_beam:
      if boolean == False:
        all_done = False
    
    if all_done:
      break

  sorted_beam = sorted(start_beam, key = lambda tup: tup[0] , reverse=True)

  sentences = []
  new_probs = []
  for s in sorted_beam:
    sentences.append(s[1])
    new_probs.append(s[0])
   
  
  
    
  
  
    








  return sentences, new_probs


sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> trump", beam=10, sampler=top_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)
print("#########################\n")

sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> biden", beam=10, sampler=top_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)
print("#########################\n")

sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> trump", beam=10, sampler=sample_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)
print("#########################\n")

sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> biden", beam=10, sampler=sample_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)

['<BOS1>', '<BOS2>', 'trump', 'eyes', 'new', 'unproven', 'coronavirus', 'treatment', 'URL', '<EOS>'] 0.00021893147502903603
['<BOS1>', '<BOS2>', 'trump', 'eyes', 'new', 'unproven', 'coronavirus', 'cure', 'URL', '<EOS>'] 0.0001719607222046247
['<BOS1>', '<BOS2>', 'trump', 'eyes', 'new', 'unproven', 'virus', 'cure', 'promoted', 'by', 'mypillow', 'ceo', 'over', 'unproven', 'therapeutic', 'URL', '<EOS>'] 9.773272077557522e-05
['<BOS1>', '<BOS2>', 'trump', 'eyes', 'new', 'unproven', 'coronavirus', 'therapeutic', 'mypillow', 'creator', 'over', 'unproven', 'therapeutic', 'URL', '<EOS>'] 8.212549111137046e-05
['<BOS1>', '<BOS2>', 'trump', 'eyes', 'new', 'unproven', 'virus', 'cure', 'promoted', 'by', 'mypillow', 'ceo', 'over', 'unproven', 'therapeutic', 'URL', 'via', '@USER', '<EOS>'] 7.432226908194607e-06
['<BOS1>', '<BOS2>', 'trump', 'eyes', 'new', 'unproven', 'virus', 'cure', 'promoted', 'by', 'mypillow', 'ceo', 'over', 'unproven', 'and', 'dangerous', 'covid-19', 'treatment', 'URL', '<EOS>']

# 2. Semantic Parsing

In this part, you are going to build your own virtual assistant! We will be developing two modules: an intent classifier and a slot filler.

In [10]:
!ls "/content/drive/My Drive/nl2ds/semantic-parser"
parser_files = "/content/drive/My Drive/nl2ds/semantic-parser"

test_answers.txt  test_questions.txt  train_questions_answers.txt


In [11]:
import json

train_data = []
for line in open(f'{parser_files}/train_questions_answers.txt'):
    train_data.append(json.loads(line))

# print a few examples
for i in range(5):
    print(train_data[i])
    print("-"*80)

{'question': 'Add an album to my Sylvia Plath playlist.', 'intent': 'AddToPlaylist', 'slots': {'music_item': 'album', 'playlist_owner': 'my', 'playlist': 'Sylvia Plath'}}
--------------------------------------------------------------------------------
{'question': 'add Diarios de Bicicleta to my la la playlist', 'intent': 'AddToPlaylist', 'slots': {'playlist': 'Diarios de Bicicleta', 'playlist_owner': 'my', 'entity_name': 'la la'}}
--------------------------------------------------------------------------------
{'question': 'book a table at a restaurant in Lucerne Valley that serves chicken nugget', 'intent': 'BookRestaurant', 'slots': {'restaurant_type': 'restaurant', 'city': 'Lucerne Valley', 'served_dish': 'chicken nugget'}}
--------------------------------------------------------------------------------
{'question': 'add iemand als jij to my playlist named In The Name Of Blues', 'intent': 'AddToPlaylist', 'slots': {'entity_name': 'iemand als jij', 'playlist_owner': 'my', 'playlist'

In [12]:
test_questions = []
for line in open(f'{parser_files}/test_questions.txt'):
    test_questions.append(json.loads(line))

test_answers = []
for line in open(f'{parser_files}/test_answers.txt'):
    test_answers.append(json.loads(line))

# print a few examples
for i in range(5):
    print(test_questions[i])
    print(test_answers[i])
    print("-"*80)

Add an artist to Jukebox Boogie Rhythm & Blues
{'intent': 'AddToPlaylist', 'slots': {'music_item': 'artist', 'playlist': 'Jukebox Boogie Rhythm & Blues'}}
--------------------------------------------------------------------------------
Will it be rainy at Sunrise in Ramey Saudi Arabia?
{'intent': 'GetWeather', 'slots': {'condition_description': 'rainy', 'timeRange': 'Sunrise', 'city': 'Ramey', 'country': 'Saudi Arabia'}}
--------------------------------------------------------------------------------
Weather in two hours  in Uzbekistan
{'intent': 'GetWeather', 'slots': {'timeRange': 'in two hours', 'country': 'Uzbekistan'}}
--------------------------------------------------------------------------------
Will there be a cloud in VI in 14 minutes ?
{'intent': 'GetWeather', 'slots': {'condition_description': 'cloud', 'state': 'VI', 'timeRange': 'in 14 minutes'}}
--------------------------------------------------------------------------------
add nuba to my Metal Party playlist
{'intent': 

## Problem 2.1: Keyword-based intent classifier

In this part, you will build a keyword-based intent classifier. For each intent, come up with a list of keywords that are important for that intent, and then classify a given question into an intent. If an input question matches multiple intents, pick the best one. If it does not match any keyword, return None.

Caution: You are allowed to look at training questions and answers to come up with a set of keywords, but it is a bad practice to look at test answers. 

In [13]:
# List of all intents
intents = set()
for example in train_data:
    intents.add(example['intent'])
print(intents)

{'AddToPlaylist', 'GetWeather', 'BookRestaurant'}


In [14]:
#looking at the training data for having a better idea of the keywords
for i in range(20):
    print(train_data[i])
    print("-"*80)

{'question': 'Add an album to my Sylvia Plath playlist.', 'intent': 'AddToPlaylist', 'slots': {'music_item': 'album', 'playlist_owner': 'my', 'playlist': 'Sylvia Plath'}}
--------------------------------------------------------------------------------
{'question': 'add Diarios de Bicicleta to my la la playlist', 'intent': 'AddToPlaylist', 'slots': {'playlist': 'Diarios de Bicicleta', 'playlist_owner': 'my', 'entity_name': 'la la'}}
--------------------------------------------------------------------------------
{'question': 'book a table at a restaurant in Lucerne Valley that serves chicken nugget', 'intent': 'BookRestaurant', 'slots': {'restaurant_type': 'restaurant', 'city': 'Lucerne Valley', 'served_dish': 'chicken nugget'}}
--------------------------------------------------------------------------------
{'question': 'add iemand als jij to my playlist named In The Name Of Blues', 'intent': 'AddToPlaylist', 'slots': {'entity_name': 'iemand als jij', 'playlist_owner': 'my', 'playlist'

In [15]:
def predict_intent_using_keywords(question):
  #includes some music genres for better accuracy, will be tried without them as well
  keywords_playlist = ['album' , 'Album' , 'add' , 'Add' , 'playlist'  , 'Playlist' , 'put' , 'Put' , 'tune' ,'Tune', 'song' , 'Song', 'Songs','songs', 'music' , 'Music' ,'musics','Musics', 'named' , 'rock & roll' , 'techno' , 'Techno' , 'Pop' , 'Rap' , 'pop' , 'rap' , 'Hip-Hop' , 'hiphop' , 'HipHop', 'indie' , 'Indie' , 'Electronic' , 'Organica' , 'organica' , 'track' , 'tracks' , 'Track' , 'Tracks']
  keywords_weather = ['weather' , 'Weather' , 'What', 'position','Position', 'area', 'Area', 'rain', 'Rain' , 'snow' , 'Snow' , 'cold' , 'Cold' , 'colder' , 'Colder' , 'warm' , 'warmer' , 'Warm' , 'Warmer' , 'sun' , 'Sun' , 'Sunny' , 'sunny' , 'winter' , 'Winter' , 'Summer' , 'summer', 'heat' , 'Heat', 'wave' , 'Wave', 'Spring' , 'spring' , 'fall' , 'Fall',
                      'Jan.' , 'Jan' , 'january' , 'January' ,'jan' , 'jan.' , 
                      'Feb.', 'feb.' , 'Feb' ,'feb' , 'February' , 'february' , 
                      'Mar.' , 'mar.' , 'Mar' , 'mar' , 'March' , 'march' ,
                      'Apr.' , 'apr.' , 'Apr' , 'apr' , 'April' , 'april' , 
                      'May.' , 'may.' , 'May', 'may',
                      'June' , 'june',
                      'July' , 'july',
                      'Aug.' , 'aug.' , 'Aug' , 'aug' , 'August' , 'august',
                      'Sept.', 'sept.' , 'Sept' , 'sept', 'September' , 'september',
                      'Oct.' , 'oct.' , 'Oct' , 'oct' , 'October' , 'october',
                      'Nov.' , 'nov.' , 'Nov' , 'nov' , 'November' , 'november',
                      'Dec.' , 'dec.' , 'Dec' , 'dec' , 'December' , 'december']
  keywords_book = ['book' , 'Book' , 'table' , 'Table' , 'chicken' , 'Chicken' , 'beef' , 'Beef', 'meat' , 'Meat' , 'restaurant' , 'Restaurant' , 'cafe' , 'Cafe' , 'reservation' , 'Reservation' , 'reserve' , 'Reserve' ,'cafeteria', 'Cafeteria' , 'serve' , 'serves', 'Vine' , 'vine', 'coffee' , 'Coffee' , 'spring' , 'Spring' , 'Fall' , 'fall']

  score_play = 0
  score_weather = 0
  score_book = 0
  dummy=0
  for token in question.split():
    if token in keywords_playlist:
      score_play += 1
    if token in keywords_weather:
      score_weather += 1
    if token in keywords_book:
      score_book += 1
    
  
  if score_play > score_weather and score_play > score_book:
    return 'AddToPlaylist'
  if score_weather > score_play and score_weather > score_book:
    return 'GetWeather'
  if score_book > score_weather and score_book > score_play:
    return 'BookRestaurant'
  
  if score_play == 0 and score_weather ==0 and score_book == 0:
    return None
  
  return None

In [16]:
from collections import Counter

'''Gives intent wise accuracy of your model'''
def evaluate_intent_accuracy(prediction_function_name):
  correct = Counter()
  total = Counter()
  for i in range(len(test_questions)):
    q = test_questions[i]
    gold_intent = test_answers[i]['intent']
    if prediction_function_name(q) == gold_intent:
      correct[gold_intent] += 1
    total[gold_intent] += 1
  for intent in intents:
    print(intent, correct[intent]/total[intent], total[intent])
    
# Evaluating the intent classifier. 
# In our implementation, a simple keyword based classifier has achieved an accuracy of greater than 65 for each intent
evaluate_intent_accuracy(predict_intent_using_keywords)

AddToPlaylist 1.0 100
GetWeather 0.67 100
BookRestaurant 0.95 100


## Problem 2.2: Statistical intent classifier

Now, let's build a statistical intent classifier. Instead of making use of keywords like what you did above, you will first extract features from a given input question. In order to build a feature representation for a given sentence, make use of word2vec embeddings of each word and take an average to represent the sentence. Then train a logistic regression. Feel free to use any libraries you like.

In [17]:
import nltk
nltk.download('word2vec_sample')

[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.


True

In [18]:
from nltk.data import find
import gensim

word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [240]:
import pandas as pd
from nltk.tokenize import word_tokenize
nltk.download('punkt')

Train_data = pd.DataFrame(train_data , columns=['question' , 'intent' , 'slots'] )

tokenized_q = []
i = 0

for row in Train_data['question']:
  tokenized_sentence = word_tokenize(row)
  new_row = [str(word) for word in tokenized_sentence if word in word2vec_model.vocab]
  
  tokenized_q.append(new_row)
  i += 1

Train_data['tokenized_questions'] = tokenized_q
#print(Train_data['tokenized_questions'].head(5))
#print(tokenized_q[0:2])
y_train = Train_data['intent']
tokenized_q = np.array(tokenized_q)

X_train = []
for sentence in tokenized_q:
  
  vector_entry = word2vec_model[sentence]
  X_train.append(np.mean(vector_entry, axis=0))

X_train = np.array(X_train)
print(X_train)
print(X_train[0].shape)
print(X_train[1].shape)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


[[ 0.03452644  0.02010152 -0.02088016 ... -0.023274    0.05154817
  -0.00663894]
 [ 0.01267474  0.02278275  0.04106298 ... -0.02427706 -0.00235562
   0.01678254]
 [-0.01432118 -0.00722078  0.03015662 ... -0.00883467  0.01105457
  -0.00851605]
 ...
 [ 0.05842185  0.01964904  0.00868735 ... -0.04953406  0.00836307
   0.00071405]
 [ 0.01944842 -0.00018672  0.0240079  ...  0.00200638 -0.01975253
  -0.02370804]
 [ 0.04823534  0.02462134 -0.00025777 ... -0.03358248  0.03556177
  -0.0661042 ]]
(300,)
(300,)


In [263]:
'''Trains a logistic regression model on the entire training data. For an input question (x), the model learns to predict an intent (Y).'''
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline



def train_logistic_regression_intent_classifier():
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    lr.trai
    return lr


In [269]:
'''For an input question, the model predicts an intent'''
model = train_logistic_regression_intent_classifier()
def predict_intent_using_logistic_regression(question):
    
    q = question.split()
    new_q = [word for word in q if word in word2vec_model.vocab]
    
    test_sample = np.mean(word2vec_model[new_q], axis=0)
    
    return model.predict(test_sample.reshape(1,-1))
    

In [272]:
# Evaluate the intent classifier
# Your intent classifier performance will be close to 100 if you have done a good job.
evaluate_intent_accuracy(predict_intent_using_logistic_regression)

AddToPlaylist 1.0 100
GetWeather 1.0 100
BookRestaurant 1.0 100


## Problem 2.3: Slot filling

Build a slot filling model. We will just work with `AddToPlaylist` intent. Ignore other intents.

Hint: No need to rely on machine learning here. You can use ideas like maximum string matching to identify which slots are active and what thier values are. This problem's solution is intentionally left underspecified.

In [285]:
# Let's stick to one target intent.
target_intent = "AddToPlaylist"

# This intent has the following slots
target_intent_slot_names = set()
for sample in train_data:
    if sample['intent'] == target_intent:
        for slot_name in sample['slots']:
            target_intent_slot_names.add(slot_name)
print(target_intent_slot_names)


# Extract all the relevant questions of this target intent from the test examples.
target_intent_questions = [] 
for i, question in enumerate(test_questions):
    if test_answers[i]['intent'] == target_intent:
        target_intent_questions.append(question)
print(len(target_intent_questions))

{'music_item', 'entity_name', 'artist', 'playlist_owner', 'playlist'}
100


In [330]:
def extract_values():
  music_item = []
  entity_name = []
  artist = []
  playlist_owner = []
  playlist = []

  for sample in train_data:
    if sample['intent'] == target_intent:
      if 'music_item' in sample['slots']:
        music_item.append(sample['slots']['music_item'])
      if 'entity_name' in sample['slots']:
        entity_name.append(sample['slots']['entity_name'])
      if 'artist' in sample['slots']:
        artist.append(sample['slots']['artist'])
      if 'playlist_owner' in sample['slots']:
        playlist_owner.append(sample['slots']['playlist_owner'])
      if 'playlist' in sample['slots']:
        playlist.append(sample['slots']['playlist'])


  return  set(music_item), set(entity_name), (artist), set(playlist_owner), set(playlist)

music_item, entity_name, artist, playlist_owner, playlist = extract_values()



print(artist)

{'Manuelita la tortuga', 'warpaint', 'Metal Church', 'Kick Over the Traces', 'confessions', 'Falling Stars', 'The Greyest of Blue Skies', 'Transmission', 'moon river', 'Games', 'stefanie', 'heresy and the hotel choir', 'Hello Central, Give Me Heaven', 'List of Rush instrumentals', 'od mene se odvikavaj', 'Driven to Tears', 'once bitten twice bitten', 'sonntag', 'Strong', 'Stuck on Nothing', 'The Green Book', 'Kids of the Black Hole', 'kan', 'Georgetown University Alma Mater', 'ojuelegba', 'woman of the world', 'Not Fade Away', 'coming back to life', 'faccetta nera', 'Clapton Chronicles The Best of Eric Clapton', 'Corrina, Corrina', 'un jour dans notre vie', 'hammer', 'crying, waiting, hoping', 'As I Was Going to St Ives', 'Every Song Is a Cry for Love', 'cut the world', 'morton feldman piano and string quartet', 'A Journal of the Plague Year', 'Happy Holidays', 'go', 'the second three years', 'Indocumentado', 'A-Hunting We Will Go', 'vera', 'Tranquility', 'all the years', 'Give Us Rest

In [294]:
s = 'I love you'
s.find('I love')

0

In [341]:
def initialize_slots():
    slots = {}
    for slot_name in target_intent_slot_names:
        slots[slot_name] = None
    return slots

def predict_slot_values(question):
    question = question.lower()
    slots = initialize_slots()  
    
    for slot_name in target_intent_slot_names:
      
      if slot_name == 'music_item':
        for value in music_item:
          if question.find(str(value)) != -1:
            slots[slot_name] = value 
            
          
     
     
      elif slot_name == 'entity_name':
        
        for value in entity_name:
          if question.find(str(value).lower()) != -1 :
            slots[slot_name] = value 
            
            
      elif slot_name == 'artist':
        
        for value in artist:
          if question.find(str(value).lower()) != -1:
            slots[slot_name] = value 
            
          
      elif slot_name == 'playlist_owner':
        
        for value in playlist_owner:
          if question.find(value.lower()) != -1:
            slots[slot_name] = value
             
            
      
      else:
        
        for value in playlist:
          if question.find(value.lower()) != -1:
            slots[slot_name] = value 
            
            
        
      
    return slots

def evaluate_slot_prediction_recall(slot_prediction_function):
    correct = Counter()
    total = Counter()
    # predict slots for each question
    for i, question in enumerate(target_intent_questions):
        i = test_questions.index(question) # This line is added after the assignment release
        gold_slots = test_answers[i]['slots']
        predicted_slots = slot_prediction_function(question)
        for name in target_intent_slot_names:
            if name in gold_slots:
                total[name] += 1.0
                if predicted_slots.get(name, None) != None and predicted_slots.get(name).lower() == gold_slots.get(name).lower(): # This line is updated after the assignment release
                    correct[name] += 1.0
    for name in target_intent_slot_names:
        print(f"{name}: {correct[name] / total[name]}")


# Our reference implementation got these numbers. You can ask others on Slack what they got.
# music_item 1.0
# playlist 0.67
# artist  0.021739130434782608
# playlist_owner 0.9444444444444444
# entity_name 0.1111111111111111
print("Slot accuracy for your slot prediction model")
evaluate_slot_prediction_recall(predict_slot_values)


Slot accuracy for your slot prediction model
music_item: 1.0
entity_name: 0.05555555555555555
artist: 0.13043478260869565
playlist_owner: 0.9444444444444444
playlist: 0.76


In [360]:


def TP(slot_name, slot_prediction_function=predict_slot_values):
  correct = 0.
  total = 0.
  for i,question in enumerate(target_intent_questions):
      pred = slot_prediction_function(question)[slot_name]
      if slot_name in test_answers[i]['slots']:
        total += 1
        if pred != None:
          correct += 1

  return correct / total

for name in target_intent_slot_names:
  print('TP:\t' , name, TP(name))




TP:	 music_item 0.5714285714285714
TP:	 entity_name 0.5
TP:	 artist 0.07142857142857142
TP:	 playlist_owner 0.6470588235294118
TP:	 playlist 0.9705882352941176


In [363]:
def FP(slot_name, slot_prediction_function=predict_slot_values):
  false = 0.
  total = 0.
  for i,question in enumerate(target_intent_questions):
      pred = slot_prediction_function(question)[slot_name]
      if slot_name not in test_answers[i]['slots']:
        total += 1
        if pred != None:
          false += 1
  
  return false / total



for name in target_intent_slot_names:
  print('FP:\t' , name, FP(name))





FP:	 music_item 0.6329113924050633
FP:	 entity_name 0.13829787234042554
FP:	 artist 0.05813953488372093
FP:	 playlist_owner 0.5060240963855421
FP:	 playlist 0.9090909090909091


In [362]:
def TN(slot_name, slot_prediction_function=predict_slot_values):
  correct = 0.
  total = 0.
  for i,question in enumerate(target_intent_questions):
      pred = slot_prediction_function(question)[slot_name]
      if slot_name not in test_answers[i]['slots']:
        total += 1
        if pred == None:
          correct += 1
  return correct/total

for name in target_intent_slot_names:
  print('TN:\t' , name, TN(name))
  


TN:	 music_item 0.3670886075949367
TN:	 entity_name 0.8617021276595744
TN:	 artist 0.9418604651162791
TN:	 playlist_owner 0.4939759036144578
TN:	 playlist 0.09090909090909091


In [366]:
# Find a false negative prediction for each slot
# Fill in your code below along with a print statement
def FP(slot_name, slot_prediction_function=predict_slot_values):
  false = 0.
  total = 0.
  for i,question in enumerate(target_intent_questions):
      pred = slot_prediction_function(question)[slot_name]
      if slot_name in test_answers[i]['slots']:
        total +=1
        if pred == None:
          false += 1
  return false/total

for name in target_intent_slot_names:
  print('TN:\t' , name, FP(name))

TN:	 music_item 0.42857142857142855
TN:	 entity_name 0.5
TN:	 artist 0.9285714285714286
TN:	 playlist_owner 0.35294117647058826
TN:	 playlist 0.029411764705882353
